In [6]:
import os
import pandas as pd
d1 = pd.read_csv(filepath)

In [2]:
d1.head()

,Review Title,Review Comment,Sentiment,Service,Source,Yext Publisher,Stars,Published Date
0,First time I've used decathlon,"First time I've used decathlon, very reasonabl...",Positive,"Price, Range / Product",TrustPilot,NaN,5,11/21/2019 - 19:23
1,Fast delivery,Fast delivery. Great product fitting in 2 min...,Positive,"Delivery, Range / Product, Customer Service",TrustPilot,NaN,5,11/21/2019 - 18:59
2,Took ages to arrive!!,Took ages to arrive!!\n,Neutral,NaN,TrustPilot,NaN,3,11/21/2019 - 18:49
3,First and last time,I’ve been looking for some indoor football boo...,Neutral,"Delivery, Range / Product, Customer Service",TrustPilot,NaN,1,11/21/2019 - 16:44
4,Poor communication,Company offered a great price and was very eff...,Positive,"Delivery, Price, Customer Service",TrustPilot,NaN,4,11/21/2019 - 13:23


In [5]:
d = d1[['Review Comment','Sentiment']]

In [10]:
d.shape

(6948, 3)

In [3]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
import nltk
def text_cleaner(text):
    stop_words = set(stopwords.words('english'))
    f_words = [w for w in nltk.word_tokenize(text) if not w in stop_words] 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    fp_words = [w for w in f_words if not w in punctuations] 
    fp_words_stem = [ps.stem(words) for words in fp_words]
    fp_sent = ' '.join(word for word in fp_words_stem)
    return fp_sent

In [ ]:
text_cleaner(d['Review Comment'][1])

In [6]:
d['Review Comment Clean'] = d['Review Comment'].apply(text_cleaner)

C:\Users\Indium Software\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(max_features=10000)
vectorizer.fit(d['Review Comment Clean'].values)
data_tfidf = vectorizer.transform(d['Review Comment Clean'])

tfidf_to_word = np.array(vectorizer.get_feature_names())

In [8]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(data_tfidf)
H = nmf.components_

In [10]:
topic_top_word = {}
for i, topic in enumerate(H):
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in tfidf_to_word[topic.argsort()[-10:]]])))

Topic 1: deliveri,experi,custom,choic,thank,good,product,price,servic,great
Topic 2: prompt,choic,all,realli,veri,quick,deliveri,price,qualiti,good
Topic 3: alway,time,effici,recommend,brilliant,prompt,thank,custom,servic,excel
Topic 4: return,item,prompt,websit,effici,use,order,easi,deliveri,quick
Topic 5: collect,decathlon,day,deliv,onlin,arriv,time,store,item,order
Topic 6: decathlon,competit,alway,cloth,fantast,excel,product,reason,price,qualiti
Topic 7: free,describ,recommend,super,item,thank,qualiti,effici,deliveri,fast
Topic 8: excel,speedi,love,decathlon,good,product,great,alway,money,valu
Topic 9: time,good,happi,deliv,realli,well,prompt,pleas,deliveri,product
Topic 10: alway,pleas,shop,recommend,purchas,friendli,happi,staff,help,veri


In [11]:
topic_list = np.array([("Topic" + " "+ str(i)) for i in range(1,11)])
for i,topic in enumerate(W[:10000,]):
    print("Headline {}: {}".format(i+1,",".join([str(x) for x in topic_list[topic.argsort()[-6:]]])))

Headline 1: Topic 10,Topic 2,Topic 1,Topic 4,Topic 6,Topic 5
Headline 2: Topic 8,Topic 10,Topic 4,Topic 1,Topic 9,Topic 7
Headline 3: Topic 1,Topic 4,Topic 2,Topic 6,Topic 9,Topic 5
Headline 4: Topic 4,Topic 6,Topic 3,Topic 7,Topic 1,Topic 5
Headline 5: Topic 2,Topic 4,Topic 7,Topic 6,Topic 1,Topic 5
Headline 6: Topic 10,Topic 7,Topic 5,Topic 1,Topic 3,Topic 8
Headline 7: Topic 9,Topic 4,Topic 3,Topic 1,Topic 5,Topic 10
Headline 8: Topic 6,Topic 7,Topic 4,Topic 1,Topic 5,Topic 8
Headline 9: Topic 1,Topic 2,Topic 6,Topic 9,Topic 4,Topic 5
Headline 10: Topic 9,Topic 1,Topic 2,Topic 8,Topic 10,Topic 4
Headline 11: Topic 10,Topic 8,Topic 1,Topic 6,Topic 9,Topic 2
Headline 12: Topic 9,Topic 10,Topic 1,Topic 5,Topic 2,Topic 6
Headline 13: Topic 7,Topic 4,Topic 8,Topic 10,Topic 5,Topic 1
Headline 14: Topic 10,Topic 1,Topic 2,Topic 9,Topic 7,Topic 4
Headline 15: Topic 4,Topic 6,Topic 9,Topic 2,Topic 8,Topic 5
Headline 16: Topic 2,Topic 8,Topic 9,Topic 5,Topic 4,Topic 10
Headline 17: Topic 2,To

In [12]:
W.shape

(6948, 10)

In [22]:
review_topics = []
for i,topic in enumerate(W[:10000,]):
    review_topics.append(topic_list[topic.argsort()[-1:]][0])
pd.Series(review_topics).value_counts()

In [27]:
pd.Series(review_topics).value_counts()

Topic 5     2205
Topic 2      597
Topic 10     586
Topic 4      574
Topic 1      544
Topic 6      530
Topic 9      514
Topic 3      506
Topic 7      453
Topic 8      439
dtype: int64

In [7]:
d2 = pd.read_csv(filepath)

In [8]:
df = pd.concat([d1,d2]).reset_index(drop=True)

In [76]:
df.to_csv(filepath)

In [33]:
df['Review Comment Clean'] = df['Review Comment'].apply(text_cleaner)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(max_features=10000)
vectorizer.fit(df['Review Comment Clean'].values)
data_tfidf = vectorizer.transform(df['Review Comment Clean'])

tfidf_to_word = np.array(vectorizer.get_feature_names())



In [35]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(data_tfidf)
H = nmf.components_

In [36]:
topic_top_word = {}
for i, topic in enumerate(H):
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in tfidf_to_word[topic.argsort()[-10:]]])))

Topic 1: shop,custom,thank,alway,fast,good,fit,price,servic,great
Topic 2: item,use,valu,stuff,all,look,realli,qualiti,fit,good
Topic 3: time,deliveri,brilliant,effici,alway,prompt,thank,custom,servic,excel
Topic 4: realli,bought,warm,use,bike,size,comfort,love,well,fit
Topic 5: function,made,worth,well,good,excel,comfort,great,money,valu
Topic 6: decent,afford,competit,nice,fantast,cloth,excel,reason,price,qualiti
Topic 7: thank,speedi,effici,item,prompt,order,easi,fast,quick,deliveri
Topic 8: quick,boot,effici,comfi,pleas,nice,warm,happi,comfort,veri
Topic 9: use,fantast,alway,recommend,rang,happi,good,reason,decathlon,product
Topic 10: easi,decathlon,time,deliv,arriv,collect,onlin,store,item,order


In [37]:
topic_list = np.array([("Topic" + " "+ str(i)) for i in range(1,11)])
for i,topic in enumerate(W[:20000,]):
    print("Headline {}: {}".format(i+1,",".join([str(x) for x in topic_list[topic.argsort()[-6:]]])))

Headline 1: Topic 2,Topic 1,Topic 9,Topic 4,Topic 6,Topic 10
Headline 2: Topic 6,Topic 8,Topic 4,Topic 1,Topic 9,Topic 7
Headline 3: Topic 7,Topic 3,Topic 2,Topic 9,Topic 4,Topic 10
Headline 4: Topic 7,Topic 8,Topic 9,Topic 1,Topic 4,Topic 10
Headline 5: Topic 9,Topic 4,Topic 6,Topic 1,Topic 7,Topic 10
Headline 6: Topic 9,Topic 10,Topic 7,Topic 1,Topic 3,Topic 5
Headline 7: Topic 7,Topic 9,Topic 4,Topic 1,Topic 10,Topic 8
Headline 8: Topic 9,Topic 4,Topic 7,Topic 1,Topic 10,Topic 5
Headline 9: Topic 9,Topic 1,Topic 2,Topic 7,Topic 4,Topic 10
Headline 10: Topic 2,Topic 1,Topic 10,Topic 4,Topic 8,Topic 7
Headline 11: Topic 8,Topic 10,Topic 1,Topic 6,Topic 9,Topic 2
Headline 12: Topic 8,Topic 9,Topic 1,Topic 10,Topic 2,Topic 6
Headline 13: Topic 7,Topic 9,Topic 8,Topic 5,Topic 10,Topic 1
Headline 14: Topic 9,Topic 10,Topic 4,Topic 1,Topic 3,Topic 7
Headline 15: Topic 9,Topic 10,Topic 6,Topic 1,Topic 2,Topic 4
Headline 16: Topic 1,Topic 4,Topic 7,Topic 9,Topic 8,Topic 10
Headline 17: Topic

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(max_features=10000, ngram_range = (1,3))
vectorizer.fit(df['Review Comment Clean'].values)
data_tfidf = vectorizer.transform(df['Review Comment Clean'])

tfidf_to_word = np.array(vectorizer.get_feature_names())


In [40]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, solver="mu")
W = nmf.fit_transform(data_tfidf)
H = nmf.components_

In [41]:
topic_top_word = {}
for i, topic in enumerate(H):
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in tfidf_to_word[topic.argsort()[-10:]]])))

Topic 1: realli good,product good,good fit,good servic,good valu,product,good product,good price,price,good
Topic 2: great valu,product great price,great servic,great product great,product great,price,product,great price,great product,great
Topic 3: qualiti,product excel,excel valu money,excel price,servic,excel valu,excel qualiti,excel product,excel servic,excel
Topic 4: get,decathlon,order,one,bought,would,size,the,use,bike
Topic 5: great,excel valu money,excel valu,great valu money,good valu money,great valu,good valu,money,valu money,valu
Topic 6: veri good servic,store,servic great,servic product,custom servic,custom,excel servic,good servic,great servic,servic
Topic 7: price,item,easi,order,product,fast deliveri,fast,quick deliveri,quick,deliveri
Topic 8: veri pleas,veri nice,veri good qualiti,veri good servic,happi,comfort,veri happi,veri comfort,veri good,veri
Topic 9: veri good qualiti,good qualiti good,qualiti good,good qualiti product,excel qualiti,qualiti price,great qualit

In [43]:
review_topics = []
for i,topic in enumerate(W[:20000,]):
    review_topics.append(topic_list[topic.argsort()[-1:]][0])
pd.Series(review_topics).value_counts()

Topic 4     7246
Topic 10    2221
Topic 7     1480
Topic 2     1463
Topic 9     1076
Topic 5     1048
Topic 3     1028
Topic 1     1004
Topic 8      929
Topic 6      855
dtype: int64

In [4]:
import spacy

In [20]:
nlp = spacy.load('en_core_web_sm')
doc = nlp("The 22-year-old recently won ATP Challenger tournament.")

for tok in nlp(df['Review Comment'][26]):
    print(tok.text, "...", tok.dep_)

Competitive ... amod
pricing ... ROOT
and ... cc
decent ... amod
quality ... conj
for ... prep
most ... pobj
of ... prep
the ... det
gear ... pobj
and ... cc
clothing ... conj
. ... punct
Sizing ... nsubj
is ... ROOT
a ... det
bit ... npadvmod
hit ... acomp
and ... cc
miss ... conj
( ... punct
a ... det
bit ... appos
in ... prep
the ... det
smaller ... amod
side ... pobj
quite ... advmod
often ... advmod
with ... prep
the ... det
clothing ... pobj
) ... punct
. ... punct
Two ... nummod
stores ... ROOT
near ... prep
me ... pobj
with ... prep
a ... det
decent ... amod
range ... pobj
( ... punct
but ... cc
limited ... amod
sizes ... conj
in ... prep
store ... pobj
) ... punct
so ... advmod
quite ... advmod
handy ... amod
although ... mark
at ... prep
times ... pobj
, ... punct
could ... aux
do ... advcl
with ... prep
having ... pcomp
a ... det
little ... npadvmod
more ... amod
staff ... dobj
to.deal.with ... prep
enquiries ... pobj
without ... prep
having ... pcomp
to ... aux
search ... x

In [48]:
for tok in doc:
    print(tok.text, "...", tok.dep_)

The ... det
22-year ... amod
- ... punct
old ... nsubj
recently ... advmod
won ... ROOT
ATP ... compound
Challenger ... compound
tournament ... dobj
. ... punct


In [77]:
df1 = pd.read_csv(filepath)
df1.head()

,Unnamed: 0,Review Title,Review Comment,Sentiment,Service,Source,Yext Publisher,Stars,Published Date
0,0,First time I've used decathlon,"First time I've used decathlon, very reasonabl...",Positive,"Price, Range / Product",TrustPilot,NaN,5,11/21/2019 - 19:23
1,1,Fast delivery,Fast delivery. Great product fitting in 2 min...,Positive,"Delivery, Range / Product, Customer Service",TrustPilot,NaN,5,11/21/2019 - 18:59
2,2,Took ages to arrive!!,Took ages to arrive!!\n,Neutral,NaN,TrustPilot,NaN,3,11/21/2019 - 18:49
3,3,First and last time,I’ve been looking for some indoor football boo...,Neutral,"Delivery, Range / Product, Customer Service",TrustPilot,NaN,1,11/21/2019 - 16:44
4,4,Poor communication,Company offered a great price and was very eff...,Positive,"Delivery, Price, Customer Service",TrustPilot,NaN,4,11/21/2019 - 13:23


In [21]:
for tok in nlp(df['Review Comment'][16]):
    print(tok.text, "...", tok.dep_)

Comfy ... nsubj
and ... cc
reasonably- ... conj
priced ... ROOT
waterproof ... amod
Hiking ... compound
boots ... dobj
. ... punct
My ... poss
son ... nsubj
found ... ROOT
them ... nsubj
comfortable ... ccomp
to ... aux
wear ... advcl
all ... det
day ... dobj
in ... prep
cold ... amod
and ... cc
wet ... conj
weather ... pobj
. ... punct
Pleased ... ROOT
with ... prep
purchase ... pobj
. ... punct

 ... 


In [81]:
corp = str()
for i in range(len(df['Review Comment'])):
    corp += (' ')+df['Review Comment'][i]

In [93]:
df['Review Comment'][2]

'Took ages to arrive!!\n'